# SQL and BigQuery Practice Notebook

The purpose of this notebook is to keep myself sharp when it comes to accessing APIs and examining BigQuery datasets.

In [6]:
from google.cloud import bigquery

To begin, we'll need a Google Cloud developer account. I already have one, but in case you need one, you can start one <a href="https://console.cloud.google.com/">here</a>. You'll also need to download <a href="https://cloud.google.com/sdk">Cloud SDK</a>. Once you have access to your Google Cloud developer account, begin a new project, click on the hamburger menu in the upper left corner > IAM & Admin > Service Accounts, then create a service account. Name your project > Create > Continue. Click the name of the service account, and scroll down to Add Key, and create a new key. Choose the JSON option and download the key file into your local project folder. 

You'll need to provide your credentials from the key file to access the BigQuery database. You can set the credentials on the command line (<a href="https://cloud.google.com/docs/authentication/getting-started#auth-cloud-implicit-python">instructions</a>) but I kept getting errors, so another option is to set it as I did in the cell below.

In [22]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/Users/Allie/Documents/DS-Projects/SQL-BigQuery-Practice/Practice-Project-7d7e549fec37.json"

print('Credentials from environ: {}'.format(
    os.environ.get('GOOGLE_APPLICATION_CREDENTIALS')))

Credentials from environ: /Users/Allie/Documents/DS-Projects/SQL-BigQuery-Practice/Practice-Project-7d7e549fec37.json


In [23]:
# Creating a client object
client = bigquery.Client()

This project will use a public dataset of Chicago Crime. We need to refer to the dataset and fetch it with an API request.

In [32]:
# Reference to the dataset
dataset_ref = client.dataset("chicago_crime", project="bigquery-public-data")

# API request
dataset = client.get_dataset(dataset_ref)

Let's check out the tables:

In [33]:
tables = list(client.list_tables(dataset))
print("Num. of tables:", len(tables))
for table in tables:
    print(table.table_id)

Num. of tables: 1
crime


We also need to fetch the tables from the dataset:

In [34]:
# Referecing the 'crime' table from the list above
table_ref = dataset_ref.table("crime")

# API request
table = client.get_table(table_ref)

Let's check out the schema of the table:

In [35]:
table.schema

[SchemaField('unique_key', 'INTEGER', 'REQUIRED', 'Unique identifier for the record.', (), None),
 SchemaField('case_number', 'STRING', 'NULLABLE', 'The Chicago Police Department RD Number (Records Division Number), which is unique to the incident.', (), None),
 SchemaField('date', 'TIMESTAMP', 'NULLABLE', 'Date when the incident occurred. this is sometimes a best estimate.', (), None),
 SchemaField('block', 'STRING', 'NULLABLE', 'The partially redacted address where the incident occurred, placing it on the same block as the actual address.', (), None),
 SchemaField('iucr', 'STRING', 'NULLABLE', 'The Illinois Unifrom Crime Reporting code. This is directly linked to the Primary Type and Description. See the list of IUCR codes at https://data.cityofchicago.org/d/c7ck-438e.', (), None),
 SchemaField('primary_type', 'STRING', 'NULLABLE', 'The primary description of the IUCR code.', (), None),
 SchemaField('description', 'STRING', 'NULLABLE', 'The secondary description of the IUCR code, a s

The schema includes the name of the field, the datatype, mode which signifies whether the column allows null values, and a short description of the contents. 

We can look at the first few rows of the table in a dataframe.

In [36]:
client.list_rows(table, max_results=5).to_dataframe()

/Users/Allie/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.
  """Entry point for launching an IPython kernel.
/Users/Allie/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: PyarrowMissingWarning: Converting to a dataframe without pyarrow installed is often slower and will become unsupported in the future. Please install the pyarrow package.
  """Entry point for launching an IPython kernel.


,unique_key,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,...,ward,community_area,fbi_code,x_coordinate,y_coordinate,year,updated_on,latitude,longitude,location
0,10806370,JA105318,2014-11-01 00:00:00+00:00,037XX W 103RD ST,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,False,False,...,19,74,11,NaN,NaN,2014,2017-01-06 15:55:08+00:00,NaN,NaN,None
1,11218186,JB134850,2017-09-13 00:01:00+00:00,054XX S WENTWORTH AVE,1152,DECEPTIVE PRACTICE,ILLEGAL USE CASH CARD,ATM (AUTOMATIC TELLER MACHINE),False,False,...,3,37,11,NaN,NaN,2017,2018-01-31 15:55:34+00:00,NaN,NaN,None
2,5335062,HN193418,2004-03-01 00:01:00+00:00,034XX W 115TH ST,1754,OFFENSE INVOLVING CHILDREN,AGG SEX ASSLT OF CHILD FAM MBR,RESIDENCE,False,True,...,19,74,02,NaN,NaN,2004,2015-08-17 15:03:40+00:00,NaN,NaN,None
3,11704073,JC283535,2018-06-14 10:00:00+00:00,063XX W HUNTINGTON ST,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,APARTMENT,False,False,...,39,10,11,NaN,NaN,2018,2019-05-30 16:12:42+00:00,NaN,NaN,None
4,11720189,JC302155,2019-06-11 17:45:00+00:00,054XX S WENTWORTH AVE,0530,ASSAULT,AGGRAVATED: OTHER DANG WEAPON,GROCERY FOOD STORE,False,False,...,3,37,04A,1175887.0,1869033.0,2019,2019-06-30 15:56:27+00:00,41.795991,-87.630542,"(41.795991039, -87.630542489)"


In [ ]:
# create map of where crimes are located